## Тестирование датасета MOT20_ext

In [35]:
from src.data import MOT20ExtDataset
import numpy as np
import itertools
import functools

### Фильтрация

In [12]:
dataset = MOT20ExtDataset('data/MOT20_ext/train/MOT20-01/')
df = dataset.ground_truth
df

,frame,id,bb_left,bb_top,bb_width,bb_height,is_consider,class,visibility
0,1,1,199,813,140,268,1,1,0.83643
1,2,1,201,812,140,268,1,1,0.84015
2,3,1,203,812,140,268,1,1,0.84015
3,4,1,206,812,140,268,1,1,0.84015
4,5,1,208,812,140,268,1,1,0.84015
...,...,...,...,...,...,...,...,...,...
26642,425,90,4,1,500,813,0,11,1.00000
26643,426,90,4,1,501,812,0,11,1.00000
26644,427,90,4,1,502,812,0,11,1.00000
26645,428,90,4,1,503,811,0,11,1.00000


In [16]:
df = df[df['is_consider'] == 1]
df

,frame,id,bb_left,bb_top,bb_width,bb_height,is_consider,class,visibility
0,1,1,199,813,140,268,1,1,0.836430
1,2,1,201,812,140,268,1,1,0.840150
2,3,1,203,812,140,268,1,1,0.840150
3,4,1,206,812,140,268,1,1,0.840150
4,5,1,208,812,140,268,1,1,0.840150
...,...,...,...,...,...,...,...,...,...
21581,425,78,1488,432,52,89,1,1,0.015094
21582,426,78,1489,432,52,89,1,1,0.007547
21583,427,78,1489,432,52,89,1,1,0.001677
21584,428,78,1490,432,52,89,1,1,0.011111


In [13]:
test_visibility = 0.95

In [18]:
df = df[df['visibility'] >= test_visibility]
df

,frame,id,bb_left,bb_top,bb_width,bb_height,is_consider,class,visibility
367,30,2,413,807,119,274,1,1,0.95079
368,31,2,415,807,118,274,1,1,0.95184
369,32,2,417,807,118,274,1,1,0.95560
390,53,2,454,808,113,271,1,1,0.99861
391,54,2,455,808,113,271,1,1,1.00000
...,...,...,...,...,...,...,...,...,...
18529,425,61,969,517,61,139,1,1,1.00000
18530,426,61,970,518,61,139,1,1,1.00000
18531,427,61,972,519,61,139,1,1,1.00000
18532,428,61,973,520,61,139,1,1,1.00000


### Расчет длины

In [21]:
test_len = 1
type(test_len) == int

True

In [66]:
objects = df[df['id'] == 4]
print(objects['frame'].values)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28]


In [59]:
def aggregate(x, y):
    if (not type(x) == list):
        return [[x, y]]
    else:
        last = x[-1][-1]
        if (y - last > 1):
            x.append([y])
        else:
            x[-1].append(y)
    return x

In [67]:
functools.reduce(aggregate, objects['frame'].values)

[[1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28]]

In [68]:
for x in df['id'].unique():
    objects = df[df['id'] == x]
    frames_seq = functools.reduce(aggregate, objects['frame'].values)
    print()